In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("traffic-analysis-zrxob").project("final-year-plxuh")
version = project.version(4)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Final-year-4 in yolov8:: 100%|██████████| 39424/39424 [00:05<00:00, 7359.72it/s] 


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 14.7 MB/s eta 0:00:00


In [3]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from ultralytics import YOLO

# SpikeNet Implementation
class SpikeConv(nn.Module):
    """Spiking Convolutional Layer with Membrane Potential"""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dropout=0.5):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.dropout = nn.Dropout2d(dropout)
        self.membrane = nn.Parameter(torch.zeros(1, out_channels, 1, 1))
        self.threshold = 1.0
        
    def forward(self, x):
        mem_potential = self.membrane.repeat(x.size(0), 1, x.size(2), x.size(3))
        conv_out = self.conv(x)
        mem_potential = mem_potential + conv_out
        spike = (mem_potential >= self.threshold).float()
        mem_potential = mem_potential * (1 - spike)
        return self.dropout(spike)

class SpikeNet(nn.Module):
    """SpikeNet Neck Architecture"""
    def __init__(self, in_channels=[256, 512, 1024], out_channels=[256, 512, 1024], dropout=0.5):
        super().__init__()
        
        # Bottom-up pathway
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        # Lateral connections
        self.lateral_conv1 = SpikeConv(in_channels[2], out_channels[2], 1, dropout=dropout)
        self.lateral_conv2 = SpikeConv(in_channels[1], out_channels[1], 1, dropout=dropout)
        
        # Fusion layers
        self.fusion_conv1 = SpikeConv(out_channels[2], out_channels[2], 3, padding=1, dropout=dropout)
        self.fusion_conv2 = SpikeConv(out_channels[1], out_channels[1], 3, padding=1, dropout=dropout)
        self.fusion_conv3 = SpikeConv(out_channels[0], out_channels[0], 3, padding=1, dropout=dropout)

    def forward(self, features):
        c3, c4, c5 = features
    
        # Top-down pathway
        p5 = self.lateral_conv1(c5)
        p5_up = F.interpolate(p5, scale_factor=2, mode='nearest')
        p4 = self.lateral_conv2(c4) + self.adjust_conv(p5_up)  # Adjust channels
    
        p4_up = F.interpolate(p4, size=c3.shape[2:], mode='nearest')  # Match c3's spatial size
        p3 = c3 + p4_up  # No downsampling of c3
    
        # Bottom-up pathway
        n3 = self.fusion_conv1(p3)
        n3_up = F.interpolate(n3, scale_factor=0.5, mode='nearest')  # Downsample
        n4 = self.fusion_conv2(p4) + n3_up
    
        n4_up = F.interpolate(n4, scale_factor=0.5, mode='nearest')
        n5 = self.fusion_conv3(p5) + n4_up
    
        return [n3, n4, n5]

# YOLOv8 + SpikeNet Integration
class YOLOv8SpikeNet(YOLO):
    """Custom YOLOv8 with SpikeNet Neck"""
    def __init__(self, model='yolov8n.yaml', dropout=0.5):
        super().__init__(model)
        self.replace_neck(dropout)
    
    def replace_neck(self, dropout):
        # Replace PANet with SpikeNet in model architecture
        self.model.model[-1] = SpikeNet(
            in_channels=[256, 512, 1024],
            out_channels=[256, 512, 1024],
            dropout=dropout
        )

# Training Configuration
if __name__ == '__main__':
    # Initialize custom YOLOv8 + SpikeNet model
    model = YOLOv8SpikeNet(model='yolov8n.yaml')
    
    # Train the model
    results = model.train(
        data='/kaggle/working/Final-year-4/data.yaml',
        epochs=50,
        batch=16,
        imgsz=640,
        lr0=0.01,
        lrf=0.1,
        optimizer="SGD",
        augment=True,
        mosaic=True,
        weight_decay=0.0005,
        dropout=0.5
    )

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/Final-year-4/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, save_hybrid=False, con

100%|██████████| 755k/755k [00:00<00:00, 4.53MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/Final-year-4/train/labels... 13704 images, 1715 backgrounds, 0 corrupt: 100%|██████████| 13704/13704 [00:12<00:00, 1139.16it/s]


train: New cache created: /kaggle/working/Final-year-4/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


50 epochs completed in 2.495 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.85 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8n summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 119/119 [00:32<00:00,  3.61it/s]


                   all       3784       4729      0.694      0.698      0.713      0.478
         License_Plate       3293       3382       0.93      0.964      0.976      0.596
                   bus         57         69      0.599      0.454      0.525      0.412
                   car        200        644      0.645       0.86      0.825      0.574
             motorbike         34         52      0.734      0.385      0.429      0.182
                person        118        498      0.772      0.978      0.962      0.709
                 truck         65         84      0.484      0.548      0.561      0.392


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train
